In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import sys
nltk.download('wordnet')
import re
#from bs4 import BeautifulSoup

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [62]:
import gensim
from gensim.models import Word2Vec

model_file = (os.path.join(sys.path[0],'GoogleNews-vectors-negative300.bin'))
#wordmodel = Word2Vec(model_file)
wordmodel_new= gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True,limit =1000000)

In [291]:
cleaned_data = pd.read_csv(os.path.join(sys.path[0],'cleaned_data.csv')) # load cleaned reviews 



In [292]:
cleaned_data=cleaned_data.dropna() # Dropping reviews that have type NaN as this will cause when randomly selecting classes with a particular class
cleaned_data = cleaned_data.reset_index(drop=True) # reseting the index tso that it covers up for the dropped values

In [9]:
cleaned_data.head()

,Unnamed: 0,star_rating,review_body,bin_class,tokens
0,75,3,little awkward use great concept depending ski...,3,"['A', 'little', 'awkward', 'to', 'use', '.', '..."
1,94,3,brita hard sided water filter bottle decent wa...,3,"['The', 'Brita', 'Hard', 'Sided', 'Water', 'Fi..."
2,100,4,blender look like new centering pad,1,"['Our', 'blender', 'looks', 'like', 'new', 'wi..."
3,103,4,bought boyfriend loved,1,"['Bought', 'it', 'for', 'my', 'boyfriend', 'he..."
4,124,4,like dish color clear redcranberry color swirl...,1,"['What', 'I', 'like', 'about', 'the', 'dishes'..."


In [293]:
cleaned_data = cleaned_data.loc[cleaned_data["bin_class"] != 3] # remove neutral reviews

In [294]:
# I am selecting a smaller dataset due to hardware limitations
cleaned_data['bin_class']=cleaned_data['bin_class'].astype(int)
s1 = cleaned_data.bin_class[cleaned_data.bin_class.eq(0)].sample(40000,random_state=1).index
s2 = cleaned_data.bin_class[cleaned_data.bin_class.eq(1)].sample(40000,random_state=1).index 



cleaned_data = cleaned_data.loc[s1.union(s2)]
display(cleaned_data['bin_class'].value_counts())


1    40000
0    40000
Name: bin_class, dtype: int64

In [295]:
from nltk.tokenize import word_tokenize
cleaned_data['tokens'] = cleaned_data['review_body'].apply(word_tokenize)

In [67]:
# finding the average of each review vectors
def averageVector_wordmodel(tokens, w2v):
    all_avg = []
    x = w2v.index_to_key
    for i in tokens:
        count = 0
        flag = True
        vec_avg = [0]*300
        
        
        for word in i:
            if word in x:
                count += 1
                vec_avg = np.add(vec_avg, w2v[word])
                flag = False
        if not flag:
            vec_avg = np.divide(vec_avg, count)
        all_avg.append(vec_avg)
    return all_avg

In [69]:
allVectors_wordmodel = averageVector_wordmodel(cleaned_data['tokens'], wordmodel_new) #calling the method to find avg

In [70]:
avg_df_word = pd.DataFrame(allVectors_wordmodel) # creating a data frame to reduce time 

In [71]:
avg_df_word.to_csv(os.path.join(sys.path[0],'avg_vector_word.csv',index=False)) #saving avg vectors for reproducibilty 

In [72]:
ret_avg_df_word = pd.read_csv(os.path.join(sys.path[0],'avg_vector_word.csv')) # loading it

In [73]:
from sklearn.model_selection import train_test_split
y2_word = cleaned_data["bin_class"]
X_train2_word, X_test2_word, y_train2_word, y_test2_word = train_test_split(ret_avg_df_word, y2_word, test_size=0.2, random_state=42, stratify = y2_word)


In [76]:
from sklearn.linear_model import Perceptron
p_word = Perceptron(random_state=42)
p_word.fit(X_train2_word, y_train2_word)

Perceptron(random_state=42)

In [168]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print('Perceptron Model Pre Trained')
print('Accuracy of Test: ', accuracy_score(y_test2_word,p_word.predict(X_test2_word) ) * 100,'%')
print('Precision of Test: ',precision_score(y_test2_word, p_word.predict(X_test2_word), average='macro') *100,'%')
print('Recall of Test: ', recall_score(y_test2_word,p_word.predict(X_test2_word) ) * 100,'%')
print('F1 Score of Test: ', f1_score(y_test2_word,p_word.predict(X_test2_word) ) * 100,'%')

print('Accuracy of Train: ', accuracy_score(y_train2_word,p_word.predict(X_train2_word) ) * 100,'%')
print('Precision of Train: ',precision_score(y_train2_word, p_word.predict(X_train2_word), average='macro') *100,'%')
print('Recall of Train: ', recall_score(y_train2_word,p_word.predict(X_train2_word) ) * 100,'%')
print('F1 Score of Train: ', f1_score(y_train2_word,p_word.predict(X_train2_word) ) * 100,'%')

Perceptron Model Pre Trained
Accuracy of Test:  72.94375 %
Precision of Test:  78.57343061298279 %
Recall of Test:  95.1375 %
F1 Score of Test:  77.85791008132577 %
Accuracy of Train:  72.6140625 %
Precision of Train:  78.16382115703564 %
Recall of Train:  94.80937499999999 %
F1 Score of Train:  77.58838948916309 %


In [79]:
from sklearn.svm import LinearSVC
svm_model_word = LinearSVC(random_state=8)
svm_model_word.fit(X_train2_word, y_train2_word)

LinearSVC(random_state=8)

In [169]:
print('SVM Model Pre Trained')

print('Accuracy of Test: ', accuracy_score(y_test2_word,svm_model_word.predict(X_test2_word) ) * 100,'%')
print('Precision of Test: ',precision_score(y_test2_word, svm_model_word.predict(X_test2_word), average='macro') *100,'%')
print('Recall of Test: ', recall_score(y_test2_word,svm_model_word.predict(X_test2_word) ) * 100,'%')
print('F1 Score of Test: ', f1_score(y_test2_word,svm_model_word.predict(X_test2_word) ) * 100,'%')

print('Accuracy of Train: ', accuracy_score(y_train2_word,svm_model_word.predict(X_train2_word) ) * 100,'%')
print('Precision of Train: ',precision_score(y_train2_word, svm_model_word.predict(X_train2_word), average='macro') *100,'%')
print('Recall of Train: ', recall_score(y_train2_word,svm_model_word.predict(X_train2_word) ) * 100,'%')
print('F1 Score of Train: ', f1_score(y_train2_word,svm_model_word.predict(X_train2_word) ) * 100,'%')

SVM Model Pre Trained
Accuracy of Test:  82.15 %
Precision of Test:  82.19649173406398 %
Recall of Test:  80.25 %
F1 Score of Test:  81.80428134556574 %
Accuracy of Train:  81.9921875 %
Precision of Train:  82.04597469695031 %
Recall of Train:  79.94375000000001 %
F1 Score of Train:  81.61559444240616 %


In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\shubh\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [170]:
import torch.optim as optim
import torch.nn as nn

In [82]:
# Class defintion of Feedforward Neural Network with 2 hidden layers
class FF_NL(nn.Module):
    def __init__(self, input_val, hidden_v1, hidden_v2, output_val):
        
        super(FF_NL, self).__init__()
        self.fcl1 = nn.Linear(input_val, hidden_v1) 
        self.relu1 = nn.ReLU()
        self.fcl2 = nn.Linear(hidden_v1, hidden_v2)
        self.relu2 = nn.ReLU()
        self.fcl3 = nn.Linear(hidden_v2, output_val)  

    def forward(self, x_val):
        out_val = self.fcl1(x_val)
        out_val = self.relu1(out_val)
        out_val = self.fcl2(out_val)
        out_val = self.relu2(out_val)
        out_val = self.fcl3(out_val)

        return torch.sigmoid(out_val) # sigmoid for binary classification

In [296]:
# retriving avg vector for my model
ret_avg_df_nn_word = pd.read_csv(os.path.join(sys.path[0],'avg_vector_word.csv'))

In [297]:
# split for binary classification using Word2Vec vectors on pre trained model
y2_nn = cleaned_data["bin_class"]
X_train2_nn, X_test2_nn, y_train2_nn, y_test2_nn = train_test_split(ret_avg_df_nn_word, y2_nn, test_size=0.2, random_state=18, stratify = y2_nn)


In [229]:
# initialising variables
input_dim = 300
hidden_dim1 = 50
hidden_dim2 =10
output_dim = 2
num_epochs = 10

ff_nn_w2v_my_model = FF_NL(input_dim, hidden_dim1,hidden_dim2, output_dim)
loss_fn_est = nn.CrossEntropyLoss()
optimizer = optim.Adam(ff_nn_w2v_my_model.parameters(), lr=0.001)

In [173]:
ff_nn_w2v_my_model.train()
# Start training
for epoch in range(num_epochs):
    loss_train = 0
    for idx in range(len(X_train2_nn)):
        
        x_inp = X_train2_nn.iloc[idx]
        x_inp = torch.tensor(x_inp)
        x_inp = torch.reshape(x_inp,(1,300)).float()
       
        x_prob = ff_nn_w2v_my_model(x_inp)
        x_prob = torch.reshape(x_prob,(1,2))
        
        y_inp = torch.tensor([y_train2_nn.iloc[idx]])
        
        optimizer.zero_grad()
        loss = loss_fn_est(x_prob, y_inp)
        
        loss_train += loss.item()
        loss.backward()

        optimizer.step()
        tr+=1
        temp+=1

    
    print('Epoch:', (epoch + 1), 'loss =', '{:.6f}'.format((loss_train / len(X_train2_nn))))



Epoch: 1 loss = 0.502068
Epoch: 2 loss = 0.484728
Epoch: 3 loss = 0.478132
Epoch: 4 loss = 0.473435
Epoch: 5 loss = 0.469957
Epoch: 6 loss = 0.466483
Epoch: 7 loss = 0.464472
Epoch: 8 loss = 0.462123
Epoch: 9 loss = 0.461637
Epoch: 10 loss = 0.460838


In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []
tp = 0
sa = 0

with torch.no_grad():
    for idx in range(len(X_test2_nn)):
        
        cxv = X_test2_nn.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = ff_nn_w2v_my_model(cxv)
        vcx = torch.tensor([y_test2_nn.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,2))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

In [175]:
# method to concatenate vectors for first 10 review words
def concat_w2v(tokens, my_model):
    all_avg = []
    x = my_model.index_to_key
    for i in tokens:
        count = 0
        flag = True
        vec_avg = [0]*300
        max_10 = 0
        for word in i:
            if max_10 ==10:
                break
            if word in x:
                max_10+=1
                count += 1
                vec_avg = np.add(vec_avg, my_model[word])
        all_avg.append(vec_avg)
    return all_avg

In [258]:
all_vec10 = concat_w2v(cleaned_data['tokens'], wordmodel_new) # calling the method

In [259]:
avg_df_10 = pd.DataFrame(all_vec10) # creating df to save time

In [260]:
avg_df_10.to_csv(os.path.join(sys.path[0],'avg_df_10_rnn.csv',index=False))

In [261]:
avg_df_10 = pd.read_csv(os.path.join(sys.path[0],'avg_df_10_rnn.csv'))

In [262]:
avg_df_10.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.003418,-0.626953,0.034790,0.238159,-0.107544,0.007812,0.147217,-0.387207,0.047363,0.409424,...,-0.344360,0.261475,-0.815430,-0.156250,0.116211,-0.208496,-0.137695,-0.641602,0.183594,-0.305054
1,-0.075195,0.237305,-0.027832,0.275879,-0.035645,-0.099609,0.324707,-0.307861,0.020142,0.284668,...,-0.239136,0.307861,-0.560059,0.272461,-0.229523,0.307373,0.311035,-0.108093,0.177734,-0.134655
2,0.936523,0.699829,-0.291138,0.651245,-0.745911,1.101868,0.901489,-1.258240,0.322388,1.177856,...,-0.299866,0.428772,-0.021408,0.895020,-0.036194,0.251877,0.784302,-1.161377,-0.026367,-0.225830
3,-0.332153,0.173981,-0.652832,0.993103,-0.274902,0.045776,-0.187927,-0.804199,0.602280,0.339539,...,-0.380859,0.754791,-0.224609,0.903931,-0.015869,-0.315674,0.033203,0.500488,0.601074,0.224976
4,0.089355,1.164703,0.742264,0.346519,0.001587,0.715302,-0.136902,-1.475952,0.954590,0.164551,...,-0.844757,1.655396,-1.640381,-0.247070,-0.727966,-0.047157,-0.287521,-0.253410,-0.564453,-0.148682


In [98]:
# split for binary classification using concatenated vectors on pre trained model
y2_nn_4b = cleaned_data["bin_class"]
X_train2_nn_4b, X_test2_nn_4b, y_train2_nn_4b, y_test2_nn_4b = train_test_split(avg_df_10, y2_nn_4b, test_size=0.2, random_state=42, stratify = y2_nn_4b)


In [99]:
class FF_NL_4b(nn.Module):
    def __init__(self, input_val, hidden_v1, hidden_v2, output_val):
        
        super(FF_NL_4b, self).__init__()
        
        
        self.fcl1 = nn.Linear(input_val, hidden_v1) 
        self.relu1 = nn.ReLU()

        # Linear function 2: 500 --> 500
        self.fcl2 = nn.Linear(hidden_v1, hidden_v2)
        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # Linear function 3 (readout): 500 --> 3
        self.fcl3 = nn.Linear(hidden_v2, output_val)  

    def forward(self, x_val):
        # Linear function 1
        out_val = self.fcl1(x_val)
        out_val = self.relu1(out_val)
        out_val = self.fcl2(out_val)
        out_val = self.relu2(out_val)
        out_val = self.fcl3(out_val)

        return torch.sigmoid(out_val)

In [100]:
model = FF_NL_4b(300,50,10,2)
x = torch.randn(50,300)
print(model(x).shape)

torch.Size([50, 2])


In [101]:
# initialisng variables
input_dim = 300
hidden_dim1 = 50
hidden_dim2 =10
output_dim = 2
num_epochs = 10

ff_nn_w2v_my_model_4b = FF_NL(input_dim, hidden_dim1,hidden_dim2, output_dim)

loss_fn_est_4b = nn.CrossEntropyLoss()
optimizer_4b = optim.Adam(ff_nn_w2v_my_model_4b.parameters(), lr=0.001)

In [178]:
ff_nn_w2v_my_model_4b.train()
# Start training
for epoch in range(num_epochs):
    loss_train_4b = 0
    
    for idx in range(len(X_train2_nn_4b)):
        
        x_inp_4b = X_train2_nn_4b.iloc[idx]
        x_inp_4b = torch.tensor(x_inp_4b)
        x_inp_4b = torch.reshape(x_inp_4b,(1,300)).float()
        
        x_prob_4b = ff_nn_w2v_my_model_4b(x_inp_4b)
        x_prob_4b = torch.reshape(x_prob_4b,(1,2))
        
        y_inp_4b = torch.tensor([y_train2_nn_4b.iloc[idx]])
        
        optimizer_4b.zero_grad()
        loss_4b = loss_fn_est_4b(x_prob_4b, y_inp_4b)
        
        loss_train_4b += loss_4b.item()
        loss_4b.backward()

        optimizer_4b.step()
    
    print('Epoch:', (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_4b / len(X_train2_nn_4b))))



Epoch: 1 loss = 0.505935
Epoch: 2 loss = 0.503346
Epoch: 3 loss = 0.500717
Epoch: 4 loss = 0.498564
Epoch: 5 loss = 0.497481
Epoch: 6 loss = 0.497930
Epoch: 7 loss = 0.498317
Epoch: 8 loss = 0.497262
Epoch: 9 loss = 0.499047
Epoch: 10 loss = 0.495007


In [301]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_nn_4b)):
        
        cxv = X_test2_nn_4b.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = ff_nn_w2v_my_model_4b(cxv)
        vcx = torch.tensor([y_test2_nn_4b.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,2))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      8000
           1       0.81      0.71      0.76      8000

    accuracy                           0.77     16000
   macro avg       0.78      0.77      0.77     16000
weighted avg       0.78      0.77      0.77     16000



In [122]:
# loading cleaned data which has all the classes i.e neutral class as well
cleaned_data_multi = pd.read_csv(os.path.join(sys.path[0],'cleaned_data.csv'))


In [124]:
cleaned_data_multi=cleaned_data_multi.dropna() # Dropping reviews that have type NaN as this will cause when randomly selecting classes with a particular class
cleaned_data_multi = cleaned_data_multi.reset_index(drop=True) # reseting the index tso that it covers up for the dropped values

In [125]:
cleaned_data_multi.head()
(cleaned_data_multi['bin_class'].value_counts())

0    99978
1    99921
3    49974
Name: bin_class, dtype: int64

In [126]:
# I am selecting a smaller dataset due to hardware limitations
cleaned_data_multi['bin_class']=cleaned_data_multi['bin_class'].astype(int)
s1 = cleaned_data_multi.bin_class[cleaned_data_multi.bin_class.eq(0)].sample(30000,random_state=1).index
s2 = cleaned_data_multi.bin_class[cleaned_data_multi.bin_class.eq(1)].sample(30000,random_state=1).index 
s3 = cleaned_data_multi.bin_class[cleaned_data_multi.bin_class.eq(3)].sample(30000,random_state=1).index 

cleaned_data_multi = cleaned_data_multi.loc[s1.union(s2).union(s3)]
display(cleaned_data_multi['bin_class'].value_counts())


3    30000
1    30000
0    30000
Name: bin_class, dtype: int64

In [127]:
from nltk.tokenize import word_tokenize
cleaned_data_multi['tokens'] = cleaned_data_multi['review_body'].apply(word_tokenize)

In [120]:
cleaned_data_multi.head(5)

,Unnamed: 0,star_rating,review_body,bin_class,tokens
0,75,3,little awkward use great concept depending ski...,3,"[little, awkward, use, great, concept, dependi..."
3,103,4,bought boyfriend loved,1,"[bought, boyfriend, loved]"
5,149,3,work dent,3,"[work, dent]"
6,165,3,fist set cracked ok,3,"[fist, set, cracked, ok]"
11,275,1,product work got,0,"[product, work, got]"


In [128]:
# calling method to find avg of vectors which will also have neutral review vectors
all_avg_vec_multi = averageVector_wordmodel(cleaned_data_multi['tokens'], wordmodel_new)

In [133]:
avg_df_multi = pd.DataFrame(all_avg_vec_multi) # converting to df to save time

In [134]:
avg_df_multi.to_csv(os.path.join(sys.path[0],'avg_vector_word_multi.csv',index=False)) # saving as csv

In [135]:
ret_avg_df_word_multi = pd.read_csv(os.path.join(sys.path[0],'avg_vector_word_multi.csv')) # loading file

In [136]:
# Class defintion of Feedforward Neural Network with 2 hidden layers multiclass classification
class FF_NL_multi(nn.Module):
    def __init__(self, input_val, hidden_v1, hidden_v2, output_val):
        
        super(FF_NL_multi, self).__init__() 
        self.fcl1 = nn.Linear(input_val, hidden_v1) 
        self.relu1 = nn.ReLU()
        self.fcl2 = nn.Linear(hidden_v1, hidden_v2)
        self.relu2 = nn.ReLU()
        self.fcl3 = nn.Linear(hidden_v2, output_val)  

    def forward(self, x_val):

        out_val = self.fcl1(x_val)
        out_val = self.relu1(out_val)
        out_val = self.fcl2(out_val)
        out_val = self.relu2(out_val)
        out_val = self.fcl3(out_val)

        return torch.softmax(out_val,dim=1) # softmax for ternery classification

In [140]:
# split for multiclass classification using Word2Vec vectors on pre trained model
y2_nn_multi = cleaned_data_multi["bin_class"]
X_train2_nn_multi, X_test2_nn_multi, y_train2_nn_multi, y_test2_nn_multi = train_test_split(ret_avg_df_word_multi, y2_nn_multi, test_size=0.2, random_state=42, stratify = y2_nn_multi)


In [279]:
# initialising variables 
input_dim = 300
hidden_dim1 = 50
hidden_dim2 =10
output_dim = 3  # 3 for neutral class
num_epochs = 50

ff_nn_w2v_my_model_multi = FF_NL_multi(input_dim, hidden_dim1,hidden_dim2, output_dim)

loss_fn_est_multi = nn.CrossEntropyLoss()
optimizer_multi = optim.Adam(ff_nn_w2v_my_model_multi.parameters(), lr=0.0001)

In [182]:
# method to return torch tensor for target class 3, since we have class value as 3 and pytorch takes linear class values as 0,1,2 and so on
def modify_y_inp(label):
    if label[0] == 0:
        return torch.tensor([0])
    elif label[0] == 1:
        return torch.tensor([1])
    else:
        return torch.tensor([2])

In [184]:
ff_nn_w2v_my_model_multi.train()
# Start training
for epoch in range(num_epochs):
    loss_train_multi = 0
    
    for idx in range(len(X_train2_nn_multi)):
        
        x_inp = X_train2_nn_multi.iloc[idx]
        x_inp = torch.tensor(x_inp)
        x_inp = torch.reshape(x_inp,(1,300)).float()
    
        x_prob = ff_nn_w2v_my_model_multi(x_inp)
        x_prob = torch.reshape(x_prob,(1,3))
        
        y_inp = modify_y_inp([y_train2_nn_multi.iloc[idx]])
        
        optimizer_multi.zero_grad()
        loss_multi = loss_fn_est_multi(x_prob, y_inp)
        
        loss_train_multi += loss_multi.item()
        loss_multi.backward()

        optimizer_multi.step()
       
    print('Epoch:', (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_multi / len(X_train2_nn_multi))))



Epoch: 1 loss = 0.854068
Epoch: 2 loss = 0.852083
Epoch: 3 loss = 0.851135
Epoch: 4 loss = 0.850527
Epoch: 5 loss = 0.849631
Epoch: 6 loss = 0.849350
Epoch: 7 loss = 0.848820
Epoch: 8 loss = 0.847964
Epoch: 9 loss = 0.847694
Epoch: 10 loss = 0.846760


In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_nn_multi)):
        
        cxv = X_test2_nn_multi.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = ff_nn_w2v_my_model_multi(cxv)
        vcx = modify_y_inp([y_test2_nn_multi.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,3))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

In [157]:
all_vec10_multi = concat_w2v(cleaned_data_multi['tokens'], wordmodel_new) # calling the concat method with neutral class as well

In [146]:
avg_df_10_multi = pd.DataFrame(all_vec10_multi) # creating df to save time

In [271]:
avg_df_10_multi.to_csv(os.path.join(sys.path[0],'avg_df_rnn_word.csv',index=False))

In [272]:
avg_df_10_multi = pd.read_csv(os.path.join(sys.path[0],'avg_df_rnn_word.csv'))

In [284]:
# split for ternary classification using concatenated vectors on pre trained model
y2_nn_4b_multi  = cleaned_data_multi["bin_class"]
X_train2_nn_4b_multi , X_test2_nn_4b_multi , y_train2_nn_4b_multi , y_test2_nn_4b_multi  = train_test_split(avg_df_10_multi, y2_nn_4b_multi , test_size=0.2, random_state=8, stratify = y2_nn_4b_multi)


In [198]:
# initialising variables 
input_dim = 300
hidden_dim1 = 50
hidden_dim2 =10
output_dim = 3 # 3 for neutral class
num_epochs = 10

ff_nn_w2v_my_model_4b_multi  = FF_NL_multi(input_dim, hidden_dim1,hidden_dim2, output_dim)

loss_fn_est_4b_multi = nn.CrossEntropyLoss()
optimizer_4b_multi = optim.Adam(ff_nn_w2v_my_model_4b_multi.parameters(), lr=0.00001)

In [199]:
ff_nn_w2v_my_model_4b_multi.train()
# Start training
for epoch in range(num_epochs):
    loss_train_4b_multi = 0
    
    for idx in range(len(X_train2_nn_4b_multi)):
        
        x_inp_4b = X_train2_nn_4b_multi.iloc[idx]
        x_inp_4b = torch.tensor(x_inp_4b)
        x_inp_4b = torch.reshape(x_inp_4b,(1,300)).float()
        
        x_prob_4b = ff_nn_w2v_my_model_4b_multi(x_inp_4b)
        x_prob_4b = torch.reshape(x_prob_4b,(1,3))
        
        y_inp_4b = make_target([y_train2_nn_4b_multi.iloc[idx]])
        
        optimizer_4b_multi.zero_grad()
        loss_4b_multi = loss_fn_est_4b_multi(x_prob_4b, y_inp_4b)
        loss_train_4b_multi += loss_4b_multi.item()

        loss_4b_multi.backward()

        optimizer_4b_multi.step()
    
    
    print('Epoch:', (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_4b_multi / len(X_train2_nn_4b_multi))))



Epoch: 1 loss = 1.052076
Epoch: 2 loss = 0.997541
Epoch: 3 loss = 0.983895
Epoch: 4 loss = 0.977628
Epoch: 5 loss = 0.973853
Epoch: 6 loss = 0.971257
Epoch: 7 loss = 0.969320
Epoch: 8 loss = 0.967803
Epoch: 9 loss = 0.966570
Epoch: 10 loss = 0.965547


In [303]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_nn_4b_multi)):
        
        cxv = X_test2_nn_4b_multi.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = ff_nn_w2v_my_model_4b_multi(cxv)
        vcx = modify_y_inp([y_test2_nn_4b_multi.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,3))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

              precision    recall  f1-score   support

           0       0.56      0.65      0.60      6000
           1       0.63      0.60      0.62      6000
           2       0.50      0.44      0.47      6000

    accuracy                           0.56     18000
   macro avg       0.56      0.56      0.56     18000
weighted avg       0.56      0.56      0.56     18000



In [216]:
cleaned_data_rnn = pd.read_csv(os.path.join(sys.path[0],'cleaned_data.csv')) # load cleaned reviews 


In [217]:
cleaned_data_rnn=cleaned_data_rnn.dropna() # Dropping reviews that have type NaN as this will cause when randomly selecting classes with a particular class
cleaned_data_rnn = cleaned_data_rnn.reset_index(drop=True) # reseting the index tso that it covers up for the dropped values

In [218]:
cleaned_data_rnn = cleaned_data_rnn.loc[cleaned_data_rnn["bin_class"] != 3] # remove neutral reviews

In [220]:
cleaned_data_rnn.head()

,Unnamed: 0,star_rating,review_body,bin_class,tokens
2,100,4,blender look like new centering pad,1,"['Our', 'blender', 'looks', 'like', 'new', 'wi..."
3,103,4,bought boyfriend loved,1,"['Bought', 'it', 'for', 'my', 'boyfriend', 'he..."
4,124,4,like dish color clear redcranberry color swirl...,1,"['What', 'I', 'like', 'about', 'the', 'dishes'..."
10,254,1,loud alarm loud allvery disappointedrather spe...,0,"['This', '&', '#', '34', ';', 'LOUD', '&', '#'..."
11,275,1,product work got,0,"['My', 'product', 'did', ""n't"", 'work', 'when'..."


In [222]:
# I am selecting a smaller dataset due to hardware limitations
cleaned_data_rnn['bin_class']=cleaned_data_rnn['bin_class'].astype(int)
s1 = cleaned_data_rnn.bin_class[cleaned_data_rnn.bin_class.eq(0)].sample(40000,random_state=1).index
s2 = cleaned_data_rnn.bin_class[cleaned_data_rnn.bin_class.eq(1)].sample(40000,random_state=1).index 


cleaned_data_rnn = cleaned_data_rnn.loc[s1.union(s2)]
display(cleaned_data_rnn['bin_class'].value_counts())


1    40000
0    40000
Name: bin_class, dtype: int64

In [223]:
# finding the average of each review vectors
def process_inp_rnn(tokens, w2v):
    all_avg = []
    x = w2v.index_to_key
    df =0
    for i in tokens:
        
        vec_avg = []
        max_len = 0
        flag = False
        for word in i:
            #print('word',word)
            if max_len==50:
                flag = True
                break
            if word in x:
                max_len+=1
                vec_avg.append(w2v[word])
                #print('vec',vec_avg)
                
        if not flag:
            rem = 50 - max_len
            #print('rem',rem)
            for s in range(rem):
                vec_avg.append([0]*300)
            #print('vec1',vec_avg) 
        #df+=1
        
        all_avg.append(vec_avg)
        #print('all',all_avg) 
    return all_avg

In [ ]:
allVectors_rnn = process_inp_rnn(cleaned_data_rnn['tokens'], wordmodel_new) #calling the method to find avg

In [ ]:
all_rnn = pd.DataFrame(allVectors_rnn) # creating df to save time

In [ ]:
all_rnn.to_csv(os.path.join(sys.path[0],'all_rnn_word.csv',index=False)) #saving avg vectors for reproducibilty 

In [263]:
ret_all_rnn= pd.read_csv(os.path.join(sys.path[0],'avg_df_10_rnn.csv')) # loading it

In [299]:
# split for binary classification using concatenated vectors on pre trained model
from sklearn.model_selection import train_test_split
y2_nn_rnn = cleaned_data_rnn["bin_class"]
X_train2_rnn, X_test2_rnn, y_train2_rnn, y_test2_rnn = train_test_split(ret_all_rnn, y2_nn_rnn, test_size=0.2, random_state=16, stratify = y2_nn_rnn)


In [235]:
class Google_RNN(torch.nn.Module):
    def __init__(self):
        super(Google_RNN, self).__init__()
        self.num_lay = 1
        self.size_hid = 50
        self.my_rnn = torch.nn.RNN(300, self.size_hid, self.num_lay)
        self.fc = torch.nn.Linear(self.size_hid, 1)
        self.drop = torch.nn.Dropout(0.4)
    
    def forward(self, x):
        batch_len = x.size()
        hid_val = self.init_hidden(batch_len)
        out, hid_val = self.my_rnn(x, hid_val)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_len):
        hid_val = torch.zeros(self.num_lay, batch_len, self.size_hid)
        #print('hee')
        return hid_val



In [249]:
num_epochs = 10
rnn_my_model = Google_RNN()
print(rnn_my_model)
rnn_google_model = FF_NL(input_dim, hidden_dim1,hidden_dim2, output_dim)
loss_fn_rnn = torch.nn.CrossEntropyLoss()
optimizer_rnn = torch.optim.Adam(rnn_my_model.parameters(), lr = 0.0001)


My_RNN(
  (my_rnn): RNN(300, 50)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
)


In [266]:
rnn_google_model.train()

# Start training

for epoch in range(num_epochs):
    #print('hh')
    loss_train_rnn = 0
    
    for idx in range(len(X_train2_rnn)):
        #print('le',len(X_train2_rnn))
        x_inp_rnn = X_train2_rnn.iloc[idx]
        #print(x_inp_rnn)
        x_inp_rnn = torch.tensor(x_inp_rnn)
        x_inp_rnn = torch.reshape(x_inp_rnn,(1,300)).float()
        #print(x_inp_rnn)
        x_prob_rnn = rnn_google_model(x_inp_rnn)
        x_prob_rnn = torch.reshape(x_prob_rnn,(1,2))
        
        y_inp_rnn = torch.tensor([y_train2_rnn.iloc[idx]])
        
        optimizer_rnn.zero_grad()
        loss_rnn = loss_fn_rnn(x_prob_rnn, y_inp_rnn)
        
        loss_train_rnn += loss_rnn.item()
        loss_rnn.backward()

        optimizer_rnn.step()
    
    print('Epoch:',  (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_rnn / len(X_train2_rnn))))



Epoch: 1 loss = 0.558783
Epoch: 2 loss = 0.539190
Epoch: 3 loss = 0.534232
Epoch: 4 loss = 0.530159
Epoch: 5 loss = 0.526245
Epoch: 6 loss = 0.522152
Epoch: 7 loss = 0.518018
Epoch: 8 loss = 0.513964
Epoch: 9 loss = 0.510155
Epoch: 10 loss = 0.506666


In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []
tp = 0
sa = 0

with torch.no_grad():
    for idx in range(len(X_test2_rnn)):
        
        cxv = X_test2_rnn.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = rnn_google_model(cxv)
        vcx = torch.tensor([y_test2_rnn.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,2))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

In [ ]:
allVectors_rnn_multi = process_inp_rnn(cleaned_data_rnn['tokens'], My_model_trained) #calling the method to find avg

In [ ]:
rnn_df_multi = pd.DataFrame(allVectors_rnn_multi) # converting to df to save time

In [ ]:
rnn_df_multi.to_csv(os.path.join(sys.path[0],'rnn_vector_multi_word.csv',index=False)) # saving as csv

In [273]:
ret_rnn_multi = pd.read_csv(os.path.join(sys.path[0],'avg_df_rnn_word.csv')) # loading file

In [274]:
# split for multiclass classification using Word2Vec vectors on my trained model
y2_rnn_multi = cleaned_data_multi["bin_class"]
X_train2_rnn_multi, X_test2_rnn_multi, y_train2_rnn_multi, y_test2_rnn_multi = train_test_split(ret_rnn_multi, y2_rnn_multi, test_size=0.2, random_state=32, stratify = y2_rnn_multi)


In [275]:
class Google_RNN_multi(torch.nn.Module):
    def __init__(self):
        super(Google_RNN_multi, self).__init__()
        self.num_lay = 1
        self.size_hid = 50
        self.my_rnn = torch.nn.RNN(300, self.size_hid, self.num_lay)
        self.fc = torch.nn.Linear(self.size_hid, 1)
        self.drop = torch.nn.Dropout(0.4)
    
    def forward(self, x):
        batch_len = x.size()
        hid_val = self.init_hidden(batch_len)
        out, hid_val = self.my_rnn(x, hid_val)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_len):
        hid_val = torch.zeros(self.num_lay, batch_len, self.size_hid)
        #print('hee')
        return hid_val



In [280]:
num_epochs = 10
rnn_google_model_multi = Google_RNN_multi()
print(rnn_google_model_multi)
rnn_google_model_multi = FF_NL_multi(input_dim, hidden_dim1,hidden_dim2, output_dim)
loss_fn_rnn_multi = torch.nn.CrossEntropyLoss()
optimizer_rnn_multi = torch.optim.Adam(rnn_my_model.parameters(), lr = 0.0001)


Google_RNN_multi(
  (my_rnn): RNN(300, 50)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
)


In [282]:
rnn_google_model_multi.train()

# Start training

for epoch in range(num_epochs):
    #print('hh')
    loss_train_rnn = 0
    
    for idx in range(len(X_train2_rnn_multi)):
        #print('le',len(X_train2_rnn))
        x_inp_rnn = X_train2_nn_multi.iloc[idx]
        #print(x_inp_rnn)
        x_inp_rnn = torch.tensor(x_inp_rnn)
        x_inp_rnn = torch.reshape(x_inp_rnn,(1,300)).float()
        #print(x_inp_rnn)
        x_prob_rnn = rnn_google_model_multi(x_inp_rnn)
        x_prob_rnn = torch.reshape(x_prob_rnn,(1,3))
        
        y_inp_rnn = modify_y_inp([y_train2_rnn_multi.iloc[idx]])
        
        optimizer_rnn_multi.zero_grad()
        loss_rnn = loss_fn_rnn_multi(x_prob_rnn, y_inp_rnn)
        
        loss_train_rnn += loss_rnn.item()
        loss_rnn.backward()

        optimizer_rnn_multi.step()
    
    print('Epoch:',  (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_rnn / len(X_train2_rnn_multi))))



Epoch: 1 loss = 1.101173
Epoch: 2 loss = 1.101173
Epoch: 3 loss = 1.101173
Epoch: 4 loss = 1.101173
Epoch: 5 loss = 1.101173
Epoch: 6 loss = 1.101173
Epoch: 7 loss = 1.101173
Epoch: 8 loss = 1.101173
Epoch: 9 loss = 1.101173
Epoch: 10 loss = 1.101173


In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_rnn_multi)):
        
        cxv = X_test2_rnn_multi.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = rnn_google_model_multi(cxv)
        vcx = torch.tensor([y_test2_rnn_multi.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,3))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

In [ ]:
class My_GRU(torch.nn.Module):
    def __init__(self):
        super(My_GRU, self).__init__()
        self.num_lay = 1
        self.size_hid = 50
        self.my_rnn = torch.nn.GRU(300, self.size_hid, self.num_lay)
        self.fc = torch.nn.Linear(self.size_hid, 1)
        self.drop = torch.nn.Dropout(0.4)
    
    def forward(self, x):
        batch_len = x.size()
        hid_val = self.init_hidden(batch_len)
        out, hid_val = self.my_rnn(x, hid_val)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_len):
        hid_val = torch.zeros(self.num_lay, batch_len, self.size_hid)
        #print('hee')
        return hid_val


In [ ]:
ret_all_gru = pd.read_csv(os.path.join(sys.path[0],'avg_vector_word.csv'))

In [ ]:
# split for binary classification using concatenated vectors on pre trained model
from sklearn.model_selection import train_test_split
y2_nn_gru = cleaned_data_rnn["bin_class"]
X_train2_gru, X_test2_gru, y_train2_gru, y_test2_gru = train_test_split(ret_all_gru, y2_nn_gru, test_size=0.2, random_state=52, stratify = y2_nn_gru)


In [ ]:
num_epochs = 10
gru_my_model = My_RNN()
print(gru_my_model)
gru_my_model = FF_NL(input_dim, hidden_dim1,hidden_dim2, output_dim)
loss_fn_rnn = torch.nn.CrossEntropyLoss()
optimizer_rnn = torch.optim.Adam(rnn_my_model.parameters(), lr = 0.0001)


In [ ]:
gru_my_model.train()

# Start training

for epoch in range(num_epochs):
    #print('hh')
    loss_train_rnn = 0
    
    for idx in range(len(X_train2_gru)):
        #print('le',len(X_train2_rnn))
        x_inp_rnn = X_train2_gru.iloc[idx]
        #print(x_inp_rnn)
        x_inp_rnn = torch.tensor(x_inp_rnn)
        x_inp_rnn = torch.reshape(x_inp_rnn,(1,300)).float()
        #print(x_inp_rnn)
        x_prob_rnn = gru_my_model(x_inp_rnn)
        x_prob_rnn = torch.reshape(x_prob_rnn,(1,2))
        
        y_inp_rnn = torch.tensor([y_train2_gru.iloc[idx]])
        
        optimizer_rnn.zero_grad()
        loss_rnn = loss_fn_rnn(x_prob_rnn, y_inp_rnn)
        
        loss_train_rnn += loss_rnn.item()
        loss_rnn.backward()

        optimizer_rnn.step()
    
    print('Epoch:',  (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_rnn / len(X_train2_gru))))



In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_gru)):
        
        cxv = X_test2_gru.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = gru_my_model(cxv)
        vcx = torch.tensor([y_test2_gru.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,2))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))

In [ ]:
ret_gru_multi = pd.read_csv(os.path.join(sys.path[0],'avg_vector_word_multi.csv')) # loading file

In [ ]:
# split for multiclass classification using Word2Vec vectors on my trained model
y2_gru_multi = cleaned_data_multi["bin_class"]
X_train2_gru_multi, X_test2_gru_multi, y_train2_gru_multi, y_test2_gru_multi = train_test_split(ret_rnn_multi, y2_gru_multi, test_size=0.2, random_state=32, stratify = y2_gru_multi)


In [ ]:
class My_GRU_multi(torch.nn.Module):
    def __init__(self):
        super(My_RNN_multi, self).__init__()
        self.num_lay = 1
        self.size_hid = 50
        self.my_rnn = torch.nn.GRU(300, self.size_hid, self.num_lay)
        self.fc = torch.nn.Linear(self.size_hid, 1)
        self.drop = torch.nn.Dropout(0.4)
    
    def forward(self, x):
        batch_len = x.size()
        hid_val = self.init_hidden(batch_len)
        out, hid_val = self.my_rnn(x, hid_val)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_len):
        hid_val = torch.zeros(self.num_lay, batch_len, self.size_hid)
        #print('hee')
        return hid_val



In [ ]:
num_epochs = 10
gru_my_model_multi = My_RNN_multi()
print(gru_my_model_multi)
gru_my_model_multi = FF_NL_multi(input_dim, hidden_dim1,hidden_dim2, output_dim)
loss_fn_rnn_multi = torch.nn.CrossEntropyLoss()
optimizer_rnn_multi = torch.optim.Adam(rnn_my_model.parameters(), lr = 0.0001)


In [ ]:
gru_my_model_multi.train()

# Start training

for epoch in range(num_epochs):
    #print('hh')
    loss_train_rnn = 0
    
    for idx in range(len(X_train2_gru_multi)):
        #print('le',len(X_train2_rnn))
        x_inp_rnn = X_train2_gru_multi.iloc[idx]
        #print(x_inp_rnn)
        x_inp_rnn = torch.tensor(x_inp_rnn)
        x_inp_rnn = torch.reshape(x_inp_rnn,(1,300)).float()
        #print(x_inp_rnn)
        x_prob_rnn = gru_my_model_multi(x_inp_rnn)
        x_prob_rnn = torch.reshape(x_prob_rnn,(1,3))
        
        y_inp_rnn = modify_y_inp([y_train2_gru_multi.iloc[idx]])
        
        optimizer_rnn_multi.zero_grad()
        loss_rnn = loss_fn_rnn_multi(x_prob_rnn, y_inp_rnn)
        
        loss_train_rnn += loss_rnn.item()
        loss_rnn.backward()

        optimizer_rnn_multi.step()
    
    print('Epoch:',  (epoch + 1), 'loss =', '{:.6f}'.format((loss_train_rnn / len(X_train2_gru_multi))))



In [ ]:
# finding report on test split
from sklearn.metrics import classification_report
test_pred = []
orig_label = []

with torch.no_grad():
    for idx in range(len(X_test2_gru_multi)):
        
        cxv = X_test2_gru_multi.iloc[idx]
        cxv = torch.tensor(cxv)
        cxv = torch.reshape(cxv,(1,300)).float()
        probs_t = gru_my_model_multi(cxv)
        vcx = torch.tensor([y_test2_gru_multi.iloc[idx]])
        probs_t = torch.reshape(probs_t,(1,3))
        test_pred.append(torch.argmax(probs_t, dim=1).numpy()[0])
        orig_label.append(vcx.numpy()[0])
print(classification_report(orig_label,test_pred))